In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
uplne_vsechno = pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet"))

In [3]:
len(uplne_vsechno)

1250624

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [5]:
rocni_pocty = df.filter(pl.col('rok').is_between(1800, 2023)).group_by('rok').len().sort('rok').rename({'len':'knih'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
rocni_pocty

rok,knih
datetime[μs],u32
1800-01-01 00:00:00,1
1801-01-01 00:00:00,22
1802-01-01 00:00:00,19
1803-01-01 00:00:00,22
1804-01-01 00:00:00,34
1805-01-01 00:00:00,23
1806-01-01 00:00:00,16
1807-01-01 00:00:00,30
1808-01-01 00:00:00,28


In [6]:
rocni_strany = df.filter(pl.col('rok').is_between(1800, 2023)).sort(by="rok").unique(subset=['245_a','100_a']).group_by('rok').agg(pl.col("stran").sum()).sort('rok').with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
rocni_strany

rok,stran
datetime[μs],i64
1800-01-01 00:00:00,5
1801-01-01 00:00:00,1789
1802-01-01 00:00:00,3040
1803-01-01 00:00:00,3613
1804-01-01 00:00:00,6871
1805-01-01 00:00:00,3166
1806-01-01 00:00:00,4194
1807-01-01 00:00:00,5828
1808-01-01 00:00:00,4354


In [7]:
rocni_cetba = 8 * 60 * 365
rocni_cetba

175200

In [8]:
rocni_strany.filter(pl.col('stran') <= rocni_cetba)

rok,stran
datetime[μs],i64
1800-01-01 00:00:00,5
1801-01-01 00:00:00,1789
1802-01-01 00:00:00,3040
1803-01-01 00:00:00,3613
1804-01-01 00:00:00,6871
1805-01-01 00:00:00,3166
1806-01-01 00:00:00,4194
1807-01-01 00:00:00,5828
1808-01-01 00:00:00,4354


rocni_strany.sort(by="stran",descending=True).head(5)

In [10]:
rocni_strany.select(pl.col("stran")).tail(1) / rocni_cetba

stran
f64
10.029315


In [11]:
novinky = df.filter(pl.col('rok').is_between(1800, 2023)).filter(pl.col('stran') >= 50).sort('rok').unique(subset=["245_a","100_a"])
novinky = novinky.group_by('rok').len().with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
novinky = novinky.sort('rok').rename({'len':'novinky'})

In [12]:
novinky.sort(by="novinky",descending=True).head(5)

rok,novinky
datetime[μs],u32
2008-01-01 00:00:00,10590
2007-01-01 00:00:00,10362
2011-01-01 00:00:00,10284
2010-01-01 00:00:00,10264
2006-01-01 00:00:00,10154


In [13]:
celkem_vyslo = df.filter(pl.col('rok').is_between(1800, 2023)).filter(pl.col('stran') >= 50).group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'celkem_vyslo'}).sort('rok')

In [14]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [15]:
nepuvodni = df.filter(pl.col('rok').is_between(1800, 2023)).filter(pl.col('stran') >= 50).explode('041_h').filter(pl.col('041_h') != 'cze').group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'preklady'}).sort('rok')
nepuvodni

rok,preklady
i64,u32
1801,2
1802,3
1803,5
1804,8
1805,2
1806,3
1807,3
1808,6
1809,6


In [16]:
preklady = celkem_vyslo.join(nepuvodni, on='rok', how='left')
preklady = preklady.with_columns((pl.col('preklady') / pl.col('celkem_vyslo')).alias('podíl překladů'))
preklady = preklady.select(pl.col(['rok','podíl překladů']))
preklady = preklady.with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady

rok,podíl překladů
datetime[μs],f64
1801-01-01 00:00:00,0.285714
1802-01-01 00:00:00,0.230769
1803-01-01 00:00:00,0.384615
1804-01-01 00:00:00,0.32
1805-01-01 00:00:00,0.133333
1806-01-01 00:00:00,0.230769
1807-01-01 00:00:00,0.157895
1808-01-01 00:00:00,0.3
1809-01-01 00:00:00,0.315789


In [17]:
do_grafu = rocni_pocty.join(rocni_strany, on="rok", how="left")
do_grafu = do_grafu.join(novinky, on="rok", how="left")
do_grafu = do_grafu.with_columns((pl.col("stran") / (60 * 8 * 365.25)).alias("let čtení"))
# do_grafu = do_grafu.join(preklady, on='rok', how='left')
do_grafu = do_grafu.melt(id_vars="rok", value_vars=['knih','stran','let čtení','novinky'])
do_grafu = do_grafu.filter(pl.col('variable').is_in(['knih','novinky','podíl překladů']))
do_grafu = do_grafu.with_columns(pl.col('variable').replace_strict({'knih': 'všechny publikace', 'novinky': 'novinky nad 50 stran'}))
do_grafu

rok,variable,value
datetime[μs],str,f64
1800-01-01 00:00:00,"""všechny publikace""",1.0
1801-01-01 00:00:00,"""všechny publikace""",22.0
1802-01-01 00:00:00,"""všechny publikace""",19.0
1803-01-01 00:00:00,"""všechny publikace""",22.0
1804-01-01 00:00:00,"""všechny publikace""",34.0
1805-01-01 00:00:00,"""všechny publikace""",23.0
1806-01-01 00:00:00,"""všechny publikace""",16.0
1807-01-01 00:00:00,"""všechny publikace""",30.0
1808-01-01 00:00:00,"""všechny publikace""",28.0


In [18]:
titulek = "Vývoj knižní produkce na českém území"
podtitulek = ["Celkové počty záznamů v České národní bibliografii.","Vedle klasických knih může jít i o katalogy výstav apod."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [19]:
def ceska_cisla(value):
    if abs(value) >= 1_000_000:
        return f"{value/1_000_000:.0f} mil."
    elif abs(value) >= 1_000:
        return f"{value/1_000:.0f} tis."
    else:
        return str(int(value))

In [20]:
ceska_cisla(1000)

'1 tis.'

In [48]:
vyvoj_produkce = alt.Chart(
    do_grafu.to_pandas(),
    title=alt.Title('Vývoj knižní produkce na českém území',
                   subtitle=["Počty nemohou být zcela přesné. chybí například knihy, které nakladatelé nedodali do knihoven."]),
    width=400,
    height=100,
).mark_line(color= '#DB842F').encode(
    alt.X('rok:T', title=None, axis=alt.Axis(tickCount=5)),
    alt.Y('value:Q', title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6', format='s'), scale=alt.Scale(domainMin=1)),
    alt.Row('variable:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap'), sort=['všechny publikace','novinky nad 50 stran','podíl překladů'])
).resolve_scale(y='independent').configure_view(stroke='transparent').configure_axisX(grid=False, domain=False).resolve_axis(x="independent")

vyvoj_produkce

alt.Chart(...)

In [54]:
vyvoj_produkce.__dict__

{'_args': (),
 '_kwds': {'data':            rok              variable   value
  0   1800-01-01     všechny publikace     1.0
  1   1801-01-01     všechny publikace    22.0
  2   1802-01-01     všechny publikace    19.0
  3   1803-01-01     všechny publikace    22.0
  4   1804-01-01     všechny publikace    34.0
  ..         ...                   ...     ...
  443 2019-01-01  novinky nad 50 stran  9135.0
  444 2020-01-01  novinky nad 50 stran  8519.0
  445 2021-01-01  novinky nad 50 stran  8458.0
  446 2022-01-01  novinky nad 50 stran  8386.0
  447 2023-01-01  novinky nad 50 stran  7652.0
  
  [448 rows x 3 columns],
  'mark': MarkDef({
    color: '#DB842F',
    type: 'line'
  }),
  'align': Undefined,
  'autosize': Undefined,
  'background': Undefined,
  'bounds': Undefined,
  'center': Undefined,
  'config': Config({
    axisX: AxisConfig({
      domain: False,
      grid: False
    }),
    view: ViewConfig({
      stroke: 'transparent'
    })
  }),
  'datasets': Undefined,
  'descrip

In [56]:
vyvoj_produkce['title']['text']

'Vývoj knižní produkce na českém území'

In [22]:
import json
from src.me_to_neurazi import me_to_neurazi
with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [74]:
me_to_neurazi(vyvoj_produkce, kredity=kredity['default'], soubor="01_vyvoj_produkce")

<figure><a href="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" target="_blank"><img src="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" width="100%" alt="Graf s titulkem Vývoj knižní produkce na českém území" /></a></figure>
